Overall, this code demonstrates how to generate random points within a specified region of interest, create buffers around those points in both circular and square shapes, and visualize the results using geemap. This kind of approach is useful for random sampling of observations and can be applied to various geospatial analyses in **[Google Earth Engine (GEE)](https://earthengine.google.com/)**.The user must have GEE account to run this code.

In [ ]:
!pip install geemap
import geemap
import ee
ee.Authenticate()
ee.Initialize()

In [2]:
# geemap.ee_search()
aoi = ee.FeatureCollection('users/animesh_codebook/India_mainland') # select area of interest(aoi) for the GEE assests

In [19]:
# Visualization of the selected region
Map = geemap.Map(center = [26, 86], zoom = 4) # change the center lat lon according the aoi
Map.addLayer(aoi,  {'color': 'red'}, 'aoi')
Map

Map(center=[26, 86], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Tog…

In [30]:
# Generate random points within aoi
points = ee.FeatureCollection.randomPoints(region= aoi, points= 50, seed= 0, maxError= 1)
Map.addLayer(points)
Map


Map(bottom=460546.0, center=[21.46648854755089, 83.56716156005861], controls=(WidgetControl(options=['position…

In [23]:
# Define a function to create a circula buffer around a point
def buffer_point(feature):

    point = feature.geometry()
    buffer = point.buffer(5000) # radius of the circular buffer in meters
    circle = ee.Feature(buffer)
    return circle
# Create a square buffer around each point
cicular_buffers = points.map(buffer_point)


# Add the random points to the map as a red layer
Map.addLayer(cicular_buffers, {'color': 'blue'}, 'Cicular_Buffer')
Map

Map(bottom=460546.0, center=[21.46648854755089, 83.56716156005861], controls=(WidgetControl(options=['position…

In [24]:
cicular_buffers

Squared Buffer Area Creation

In [31]:
# colleting the Points cordinates in a list
points_coords = points.geometry().coordinates().getInfo()
points_coords[0:5]

[[80.3205380028601, 24.751017454572917],
 [92.25080124154209, 26.937924793876675],
 [72.86719057947086, 22.02007822348892],
 [74.12950276866006, 20.573665837754497],
 [85.65278362166923, 23.00673898270378]]

In [26]:
# Function for creating square buffer around the points
def create_square_buffer(coords_list):
    # Set buffer size in degree
    buffer_size = 0.1

    # Create a list to store the square buffers
    buffer_list = []

    # Iterate through the input coordinates and create a square buffer for each one
    for coords in coords_list:
        lon, lat = coords[0], coords[1]
        buffer = ee.Geometry.Rectangle(
            coords=[lon - 0.5 * buffer_size ,
                    lat - 0.5 * buffer_size ,
                    lon + 0.5 * buffer_size ,
                    lat + 0.5 * buffer_size ])
        buffer_list.append(buffer)

    # Convert the list of buffers to an EE FeatureCollection
    buffer_fc = ee.FeatureCollection(buffer_list)

    return buffer_fc

# Create a square buffer around each point
square_buffers = create_square_buffer(points_coords)

In [27]:
# Map.clear_layers()
Map.addLayer(square_buffers, {'color':'green'}, 'Squared_Buffer')
Map

Map(bottom=460546.0, center=[21.46648854755089, 83.56716156005861], controls=(WidgetControl(options=['position…

In [ ]:
square_buffers